In [13]:
from sqlmodel import select
from database import get_session
from models import Role, User

In [14]:
def insert_role(name: str, description: str):
    with get_session() as session:
        role = Role(name=name, description=description)
        session.add(role)
        session.commit()
        session.refresh(role)
        print(f"Role Added: {role}")
        return role
    
def insert_user(name:str, username: str, email: str, password: str, role_id: int):
    with get_session() as session:
        user = User(name=name, username=username, email=email, password_hash=password, role_id=role_id)
        session.add(user)
        session.commit()
        session.refresh(user)
        print(f"User Added: {user}")
        return user

In [15]:
# Insert roles
admin_role = insert_role("Admin", "Administrator with full access.")
editor_role = insert_role("Editor", "Can edit and manage content.")

# Insert users linked to roles
insert_user("Alice Johnson", "alicej", "alice@example.com", "hashed_password1", admin_role.id)
insert_user("Bob Smith", "bobsmith", "bob@example.com", "hashed_password2", editor_role.id)

Role Added: description='Administrator with full access.' id=1 name='Admin'
Role Added: description='Can edit and manage content.' id=2 name='Editor'
User Added: id=1 email='alice@example.com' role_id=1 name='Alice Johnson' username='alicej' password_hash='hashed_password1'
User Added: id=2 email='bob@example.com' role_id=2 name='Bob Smith' username='bobsmith' password_hash='hashed_password2'


User(id=2, email='bob@example.com', role_id=2, name='Bob Smith', username='bobsmith', password_hash='hashed_password2')

In [16]:
def get_user(username: str):
    with get_session() as session:
        statement = select(User).where(User.username == username)
        user = session.exec(statement).first()
        return user
    
user = get_user("alicej")

In [17]:
user

User(id=1, email='alice@example.com', role_id=1, name='Alice Johnson', username='alicej', password_hash='hashed_password1')

In [18]:
user.name

'Alice Johnson'

In [19]:
## To have access to role of user: use Eager Loading
from sqlalchemy.orm import selectinload
from sqlmodel import select

def get_user(username: str):
    with get_session() as session:
        statement = (
            select(User)
            .where(User.username == username)
            .options(selectinload(User.role))  # Eager load the role
        )
        user = session.exec(statement).first()
        return user

user = get_user("alicej")

if user and user.role:
    print(f"User: {user.name}, Role: {user.role.name}, Description: {user.role.description}")
else:
    print("User or role not found.")

User: Alice Johnson, Role: Admin, Description: Administrator with full access.
